In [ ]:
# Calculate minimum and maximum monthly values from CESM and reanalysis datasets for each basin
# Here CESM and reanalysis that have alreayd had the monthly time series extracted and saved as a timeseries in a .txt file


In [1]:
import HydroErr as he
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings(action='ignore')
from glob import glob
import os
import re
from datetime import datetime

In [2]:
# specify the directories that the datasets to be compared are saved in.
dir= '/Volumes/MCO_SSD/data/subbasin_data/CESM/et/grouped_subbasins/east_west/*.txt'
dir2 = '/Volumes/MCO_SSD/data/subbasin_data/CESM/et/*.txt'
dir5 = '/Volumes/MCO_SSD/data/subbasin_data/CESM/et/grouped_subbasins/entirebasin/*.txt'

dir3 = '/Volumes/MCO_SSD/data/subbasin_data/Livneh/et/et_mon_mean/grouped_subbasins/east_west/*.txt'
dir4 = '/Volumes/MCO_SSD/data/subbasin_data/Livneh/et/et_mon_mean/*.txt'
dir6 = '/Volumes/MCO_SSD/data/subbasin_data/Livneh/et/et_mon_mean/grouped_subbasins/entirebasin/*.txt'

In [3]:
# get list of all files for the two categories of datasets. save each of these as a list to be used in the loop
dir_list = glob(dir, recursive=True)
dir_list2 = glob(dir2, recursive=True)
dir_list3 = glob(dir3, recursive=True)
dir_list4 = glob(dir4, recursive=True)
dir_list5 = glob(dir5, recursive=True)
dir_list6 = glob(dir6, recursive=True)

In [4]:
# Basin names
# keep in this order, and add full basin at the end, because it determines the order of the plotting 

#subbasin_list= ['Arkansas.White.Red.Region','Lower.Mississippi.Region', 'Missouri.Region','Ohio.Region','Tennessee.Region','Upper.Mississippi.Region','Eastern.Mississippi.Basin.Region', 'Western.Mississippi.Basin.Region', 'Entire.Mississippi.Region'] 
subbasin_list= ['Eastern.Mississippi.Basin.Region', 'Western.Mississippi.Basin.Region', 'Entire.Mississippi.Region'] 


In [5]:
#check which files are being read in

final_filtered_dir_list = []
print(final_filtered_dir_list)
for basin in subbasin_list:
    print(basin)
    filtered_dir_list = [x for x in dir_list if basin in x] 
    #print(filtered_dir_list)
    filtered_dir_list_2 = [y for y in dir_list2 if basin in y] 
    #print(filtered_dir_list_2)
    filtered_dir_list_3 = [z for z in dir_list3 if basin in z]
    #print(filtered_dir_list_3)
    filtered_dir_list_4 = [w for w in dir_list4 if basin in w]
    filtered_dir_list_5 = [u for u in dir_list5 if basin in u]
    filtered_dir_list_6 = [v for v in dir_list6 if basin in v]
    final_filtered_dir_list = filtered_dir_list + filtered_dir_list_3 + filtered_dir_list_2 + filtered_dir_list_4 + filtered_dir_list_5 + filtered_dir_list_6

    final_filtered_dir_list = final_filtered_dir_list
    print(final_filtered_dir_list)
    

[]
Eastern.Mississippi.Basin.Region
['/Volumes/MCO_SSD/data/subbasin_data/CESM/et/grouped_subbasins/east_west/subbasin_monthly_avg_Eastern.Mississippi.Basin.Region_CESM_et_qsoil.txt', '/Volumes/MCO_SSD/data/subbasin_data/Livneh/et/et_mon_mean/grouped_subbasins/east_west/subbasin_monthly_avg_Eastern.Mississippi.Basin.Region_Livneh_et.txt']
Western.Mississippi.Basin.Region
['/Volumes/MCO_SSD/data/subbasin_data/CESM/et/grouped_subbasins/east_west/subbasin_monthly_avg_Western.Mississippi.Basin.Region_CESM_et_qsoil.txt', '/Volumes/MCO_SSD/data/subbasin_data/Livneh/et/et_mon_mean/grouped_subbasins/east_west/subbasin_monthly_avg_Western.Mississippi.Basin.Region_Livneh_et.txt']
Entire.Mississippi.Region
['/Volumes/MCO_SSD/data/subbasin_data/CESM/et/grouped_subbasins/entirebasin/subbasin_monthly_avg_Entire.Mississippi.Region_CESM_et_qsoil.txt', '/Volumes/MCO_SSD/data/subbasin_data/Livneh/et/et_mon_mean/grouped_subbasins/entirebasin/subbasin_monthly_avg_Entire.Mississippi.Region_Livneh_et.txt']


In [6]:
# Setup dataframes for calculations

df = pd.DataFrame()
df_2 = pd.DataFrame()
df_original_all = pd.DataFrame()
#df_lagcor = pd.DataFrame()

df_means_all = pd.DataFrame()
df_dfmeans = pd.DataFrame()
df_df2means = pd.DataFrame()


df_lagcor_all = pd.DataFrame()
df_peakmonths_all = pd.DataFrame()

i = 0

# loop through basins
for basin in subbasin_list:
    print(basin)
    basin_num = subbasin_list.index(basin)

    # compile directory lists to access datasets
    filtered_dir_list = [x for x in dir_list if basin in x] 
    filtered_dir_list_2 = [y for y in dir_list2 if basin in y] 
    filtered_dir_list_3 = [z for z in dir_list3 if basin in z]
    filtered_dir_list_4 = [w for w in dir_list4 if basin in w]
    filtered_dir_list_5 = [u for u in dir_list5 if basin in u]
    filtered_dir_list_6 = [v for v in dir_list6 if basin in v]
    final_filtered_dir_list = filtered_dir_list + filtered_dir_list_3 + filtered_dir_list_2 + filtered_dir_list_4 + filtered_dir_list_5 + filtered_dir_list_6

    # setup basins names and directory paths
    basin_name = basin
    directory1 = final_filtered_dir_list[i]
    directory2 = final_filtered_dir_list[i+1]
    name1 = os.path.basename(directory1)
    name2 = os.path.basename(directory2)
    print(name1)
    print(name2)
    
    # setup dataframe1  ## this could be updated to add any other datasets that will be plotted on the same y axis
    df = pd.read_csv(directory1, sep=" ", header = 0)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['DATE2'] = df['DATE'] # fix this. on the plane and don't remember the option to keep a column while setting it as index
    df['month_txt'] = df['DATE'].dt.strftime("%b")
    df = df[(df['DATE'] >='1950-01-01') & (df['DATE'] <='2005-01-01')]
    df = df.set_index('DATE')
    #df['value'] = df['value']*(2629800) # CESM units are mm/s, so multiply by seconds in a month to convert to mm. Since time step is monthly, this converts to common time step of mm/month.
    #df['value'] = (df['value'] - df['value'].mean()) / df['value'].std()  #can normalize the values to be between -1 and 1 for the correlation calcualtions below
    df['dataset'] = "CESMLME"
    df['basin'] = basin
    df_means = df.groupby('month').agg({'value': 'mean', 'month_txt':'first', 'dataset':'first', 'basin':'first'})
    df_means = df_means.reset_index()
    #df_means.insert(2,'dataset_name',name1) #name = CESM
    
    # setup dataframe2
    df_2 = pd.read_csv(directory2, sep=" ", header = 0)
    df_2['DATE'] = pd.to_datetime(df_2['DATE'])
    df_2['DATE2'] = df_2['DATE']
    df_2['month_txt'] = df_2['DATE'].dt.strftime("%b")
    df_2 = df_2[(df_2['DATE'] >='1950-01-01') & (df_2['DATE'] <='2005-01-01')]
    df_2 = df_2.set_index('DATE')
    #df_2['value'] = df_2['value']*(1000) # ERA5 units are m, so multiply by 1000 to convert to mm. Since time step is monthly, this converts to common time step of mm/month.
    #df_2['value'] = (df_2['value'] - df_2['value'].mean()) / df_2['value'].std()
    df_2['dataset'] = "ERA5"
    df_2['basin'] = basin
    df2_means = df_2.groupby('month').agg({'value': 'mean', 'month_txt':'first', 'dataset':'first', 'basin':'first'})
    df2_means = df2_means.reset_index()
    #df2_means.insert(2,'dataset_name',name3) #name = ERA5
    
    df_dfmeans = df_dfmeans.append(df_means, ignore_index = True)
    df_df2means = df_df2means.append(df2_means, ignore_index = True)
    
    df_means_all = df_means_all.append(df_means, ignore_index = True)
    df_means_all = df_means_all.append(df2_means, ignore_index = True)

print(df_means_all)

#print(df_dfmeans)
#print(df_df2means)


Eastern.Mississippi.Basin.Region
subbasin_monthly_avg_Eastern.Mississippi.Basin.Region_CESM_et_qsoil.txt
subbasin_monthly_avg_Eastern.Mississippi.Basin.Region_Livneh_et.txt
Western.Mississippi.Basin.Region
subbasin_monthly_avg_Western.Mississippi.Basin.Region_CESM_et_qsoil.txt
subbasin_monthly_avg_Western.Mississippi.Basin.Region_Livneh_et.txt
Entire.Mississippi.Region
subbasin_monthly_avg_Entire.Mississippi.Region_CESM_et_qsoil.txt
subbasin_monthly_avg_Entire.Mississippi.Region_Livneh_et.txt
    month     value month_txt  dataset                             basin
0       1  0.000007       Jan  CESMLME  Eastern.Mississippi.Basin.Region
1       2  0.000008       Feb  CESMLME  Eastern.Mississippi.Basin.Region
2       3  0.000011       Mar  CESMLME  Eastern.Mississippi.Basin.Region
3       4  0.000019       Apr  CESMLME  Eastern.Mississippi.Basin.Region
4       5  0.000031       May  CESMLME  Eastern.Mississippi.Basin.Region
..    ...       ...       ...      ...                          

In [7]:
# Set the display options to show all rows and columns
pd.set_option('display.max_rows', None)

In [8]:
# Convert the 'month' column to pandas DatetimeIndex
df_means_all['month'] = pd.to_datetime(df_means_all['month'], format='%m').dt.month

# Calculate the monthly mean values of QCHANR for each basin
monthly_mean_QCHANR = df_means_all.groupby(['dataset', 'basin', 'month'])['value'].mean().reset_index()

monthly_mean_QCHANR

,dataset,basin,month,value
0,CESMLME,Eastern.Mississippi.Basin.Region,1,0.000007
1,CESMLME,Eastern.Mississippi.Basin.Region,2,0.000008
2,CESMLME,Eastern.Mississippi.Basin.Region,3,0.000011
3,CESMLME,Eastern.Mississippi.Basin.Region,4,0.000019
4,CESMLME,Eastern.Mississippi.Basin.Region,5,0.000031
5,CESMLME,Eastern.Mississippi.Basin.Region,6,0.000041
6,CESMLME,Eastern.Mississippi.Basin.Region,7,0.000046
7,CESMLME,Eastern.Mississippi.Basin.Region,8,0.000045
8,CESMLME,Eastern.Mississippi.Basin.Region,9,0.000038
9,CESMLME,Eastern.Mississippi.Basin.Region,10,0.000027


In [10]:
# Find the index of the maximum value for each dataset and basin
max_indices = monthly_mean_QCHANR.groupby(['dataset', 'basin'])['value'].idxmax()

In [11]:
# Get the month and maximum value for each dataset and basin
result_df = monthly_mean_QCHANR.loc[max_indices, ['dataset', 'basin', 'month', 'value']]
result_df


,dataset,basin,month,value
6,CESMLME,Eastern.Mississippi.Basin.Region,7,0.000046
16,CESMLME,Entire.Mississippi.Region,5,0.000009
30,CESMLME,Western.Mississippi.Basin.Region,7,0.000047
41,ERA5,Eastern.Mississippi.Basin.Region,6,0.000049
53,ERA5,Entire.Mississippi.Region,6,0.000040
65,ERA5,Western.Mississippi.Basin.Region,6,0.000038


In [12]:
# Find the index of the minimum value for each dataset and basin
min_indices = monthly_mean_QCHANR.groupby(['dataset', 'basin'])['value'].idxmin()

In [13]:
# Get the month and maximum value for each dataset and basin
result_min_df = monthly_mean_QCHANR.loc[min_indices, ['dataset', 'basin', 'month', 'value']]
result_min_df

,dataset,basin,month,value
0,CESMLME,Eastern.Mississippi.Basin.Region,1,0.000007
12,CESMLME,Entire.Mississippi.Region,1,0.000002
24,CESMLME,Western.Mississippi.Basin.Region,1,0.000004
36,ERA5,Eastern.Mississippi.Basin.Region,1,0.000009
59,ERA5,Entire.Mississippi.Region,12,0.000006
71,ERA5,Western.Mississippi.Basin.Region,12,0.000005


In [14]:
# Reset the display options to default if needed
pd.reset_option('display.max_rows')